In [43]:
import numpy as np
from keras.datasets import boston_housing
from keras import models
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers

In [44]:
#Load data
(trainData, trainTargets), (testData, testTargets) = boston_housing.load_data()

#Get a feel
trainData.shape


(404, 13)

In [45]:
#Normalize data
mean = trainData.mean(axis=0)
trainData -= mean
std = trainData.std(axis=0)
trainData /= std

testData -= mean
testData /= std

In [46]:
def buildModel():
    model = models.Sequential()
    model.add(layers.Dense(64, activation="relu", input_shape=(trainData.shape[1],)))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(1))
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model


In [47]:
#Declare number of partitions 
k = 4

#numValSamples splits the training data into n partitions with floor division
#Evaluates to 101
numValSamples = len(trainData) // k

#However many you like
numEpochs = 500

#Empty list for scores
allScores = []

#Loop through each partition
for i in range(k):
    print("processing fold #", i)
    
    #Slice for each partition, i.e., (0 * numValSamples : 1 * numValSamples) evaluates as (0 : 101)
    #Second pass (1 * numValSamples : 2 * numValSamples) evaluates as (101 : 202) 
    #And so on
    valData = trainData[i * numValSamples: (i + 1) * numValSamples]
    
    #Repeat the same for our validation targets
    valTargets = trainTargets[i * numValSamples: (i + 1) * numValSamples]
    
    #Partition our validation data in much the same manner
    partialTrainData = np.concatenate(
        [trainData[:i * numValSamples],
         trainData[(i + 1) * numValSamples:]],
        axis = 0)
    partialTrainTargets = np.concatenate(
        [trainTargets[:i * numValSamples],
         trainTargets[(i + 1) * numValSamples:]],
        axis = 0)
    
    #Build, fit, evaluate model with mean absolute error appeneded to the empty scores list
    model = buildModel()
    history = model.fit(partialTrainData, partialTrainTargets, validation_data=(valData, valTargets), epochs = numEpochs, batch_size = 1, verbose = 0)
    maeHistory = history.history['val_mae']
    allScores.append(maeHistory)
    #valMse, valMae = model.evaluate(valData, valTargets, verbose = 0)
    #allScores.append(valMae)

#How much we're off by
np.mean(allScores)
    


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


2.571161377310753